In [2]:
import re
import numpy as np
import collections
from ase.io import read
from ase.atoms import Atoms

In [3]:
def read_atom(poscar_file, twoDmaterials_elements, substrate_elements):
    atoms = read(poscar_file)
    atoms.set_pbc([True, True, False])
    all_elements = list(set(atoms.get_chemical_symbols()))
    print(f"Elements in POSCAR: {all_elements}")

    atoms_by_element = collections.defaultdict(list)
    twoDmaterials_atoms = []
    substrate_atoms = []

    for index, atom in enumerate(atoms):
        if atom.symbol in twoDmaterials_elements:
            twoDmaterials_atoms.append((atom, index))
        elif atom.symbol in substrate_elements:
            substrate_atoms.append((atom, index))

    return twoDmaterials_atoms, substrate_atoms, atoms

In [4]:
def identify(twoDmaterials_atoms, substrate_atoms, tolerance=0.1):
    twoDmaterials_bottom_layer = []
    substrate_top_layer = []
    twoDmaterials_bottom_layer_idx = []
    substrate_top_layer_idx = []
    

    if twoDmaterials_atoms:
        twoDmaterials_zcoords = np.array([atom_obj.position[2] for atom_obj, _ in twoDmaterials_atoms])
        min_twoD_position = np.min(twoDmaterials_zcoords)

        for atom_obj, original_idx in twoDmaterials_atoms: # 遍历 (Atom对象, 原始索引) 元组
            if abs(atom_obj.position[2] - min_twoD_position) < tolerance:
                twoDmaterials_bottom_layer.append((atom_obj, original_idx)) # 存储 (Atom对象, 原始索引)
                twoDmaterials_bottom_layer_idx.append(original_idx)
    else:
        print("No 2D material atoms provided to identify bottom layer.")

    # Process substrate atoms
    if substrate_atoms:
        substrate_z_coords = np.array([atom_obj.position[2] for atom_obj, _ in substrate_atoms])
        max_substrate_z = np.max(substrate_z_coords)

        for atom_obj, original_idx in substrate_atoms: # 遍历 (Atom对象, 原始索引) 元组
            if abs(atom_obj.position[2] - max_substrate_z) < tolerance:
                substrate_top_layer.append((atom_obj, original_idx)) # 存储 (Atom对象, 原始索引)
                substrate_top_layer_idx.append(original_idx)
    else:
        print("No substrate atoms provided to identify top layer.")

    return twoDmaterials_bottom_layer_idx, substrate_top_layer_idx

In [5]:
def calculate(twoDmaterials_bottom_layer_idx, substrate_top_layer_idx, interface_system_atoms):

    closest_distances_for_each_2d_atom = []
    calculated_pair_indices = [] 
    
    # Iterate through each atom in the 2D material's bottom layer
    for two_d_atom_indices in twoDmaterials_bottom_layer_idx:
        min_dist_for_current_2d_atom = float('inf') # Initialize with a very large number
        closest_sub_atom_idx = None
        
        # Compare it to every atom in the substrate's top layer
        for sub_atom_indices in substrate_top_layer_idx:
            # Calculate the Euclidean distance between the two atoms
            distance = interface_system_atoms.get_distance(two_d_atom_indices, sub_atom_indices, mic=True)

            # Update min_dist if a shorter distance is found
            if distance < min_dist_for_current_2d_atom:
                min_dist_for_current_2d_atom = distance
                closest_sub_atom_idx = sub_atom_indices
                

        # After checking all substrate atoms for the current 2D atom, store its shortest distance
        if min_dist_for_current_2d_atom != float('inf'): # Ensure a valid distance was found
            closest_distances_for_each_2d_atom.append(min_dist_for_current_2d_atom)
            if closest_sub_atom_idx is not None:
                calculated_pair_indices.append((two_d_atom_indices, closest_sub_atom_idx))
                print(f"  - 2D atom index: {two_d_atom_indices} (Symbol: {interface_system_atoms[two_d_atom_indices].symbol}) "
                      f"-> Closest Substrate atom index: {closest_sub_atom_idx} (Symbol: {interface_system_atoms[closest_sub_atom_idx].symbol}), "
                      f"Distance: {min_dist_for_current_2d_atom:.3f} Å")
        else:
            print(f"Warning: Could not find any distance for 2D atom. This should not happen if layers are not empty.")


    if closest_distances_for_each_2d_atom:
        average_closest_distance = np.mean(closest_distances_for_each_2d_atom)
        return average_closest_distance
    else:
        print("No valid shortest distances were found for calculation.")
        return None


In [6]:
if __name__ == "__main__":
    file_out = open("result.txt",'w',encoding='utf-8')
    for g in range(20):
        for h in range(20):
            poscar_file = "POSCAR_"+str(g)+"_"+str(h)
            print(poscar_file, file = file_out)

            twoDmaterials_elements = ['Mo', 'S']
            substrate_elements = ['Au']

            twoDmaterials_atoms, substrate_atoms, interface_system_atoms = read_atom(poscar_file, twoDmaterials_elements, substrate_elements)
            if twoDmaterials_atoms is not None and substrate_atoms is not None:
                twoDmaterials_bottom_layer_idx, substrate_top_layer_idx = identify(twoDmaterials_atoms, substrate_atoms, tolerance=0.1)
                print("\nCalculating average shortest interfacial distance...")
                average_distance = calculate(twoDmaterials_bottom_layer_idx, substrate_top_layer_idx, interface_system_atoms)

                if average_distance is not None:
                    print(f"Average shortest distance from 2D material bottom layer to substrate top layer: {average_distance:.3f} Å", file = file_out)
                else:
                    print("Failed to calculate average shortest distance.", file = file_out)

            else:
                print("\nAtom categorization failed or no atoms were found for 2D material/substrate. Please check the POSCAR file and element inputs.")
file_out.close()

Elements in POSCAR: ['S', 'Au', 'Mo']

Calculating average shortest interfacial distance...
  - 2D atom index: 196 (Symbol: S) -> Closest Substrate atom index: 121 (Symbol: Au), Distance: 3.441 Å
  - 2D atom index: 198 (Symbol: S) -> Closest Substrate atom index: 149 (Symbol: Au), Distance: 3.243 Å
  - 2D atom index: 200 (Symbol: S) -> Closest Substrate atom index: 121 (Symbol: Au), Distance: 3.350 Å
  - 2D atom index: 202 (Symbol: S) -> Closest Substrate atom index: 181 (Symbol: Au), Distance: 3.262 Å
  - 2D atom index: 204 (Symbol: S) -> Closest Substrate atom index: 125 (Symbol: Au), Distance: 3.361 Å
  - 2D atom index: 206 (Symbol: S) -> Closest Substrate atom index: 153 (Symbol: Au), Distance: 3.298 Å
  - 2D atom index: 208 (Symbol: S) -> Closest Substrate atom index: 97 (Symbol: Au), Distance: 3.357 Å
  - 2D atom index: 210 (Symbol: S) -> Closest Substrate atom index: 125 (Symbol: Au), Distance: 3.403 Å
  - 2D atom index: 212 (Symbol: S) -> Closest Substrate atom index: 185 (Symb